# Stworzenie instancji testowych łamigłówki sudoku

## Załadowanie danych z kaggle'a
Plik z 3 milionami wygenerowanych łamigłówek można pobrać [stąd](https://www.kaggle.com/radcliffe/3-million-sudoku-puzzles-with-ratings).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
import json

Ładujemy dane. Brak cyfry w przykładowym problemie chcemy oznaczać zerem, więc zamieniamy od razu kropki na zera.

In [ ]:
raw_df = pd.read_csv('sudoku-3m.csv', dtype={'puzzle': str, 'solution': str})

replace_with_zeroes = lambda puzzle: puzzle.replace('.', '0')
raw_df['puzzle'] = raw_df['puzzle'].apply(replace_with_zeroes)

raw_df.head()

In [ ]:
raw_df.describe()

Rozkład liczby wskazówek i trudności łamigłówek

In [ ]:
%matplotlib inline
raw_df[['difficulty', 'clues']].hist()

## Przygotowanie zbiorów o 3 poziomach trudności

Ustawienia

In [ ]:
# ile chcemy instancji z każdego poziomu trudności
limit_per_difficulty_level = 50  

# zakresy kolumny 'difficulty' dla każdego poziomu trudności
difficulty_ranges = {  
    'easy': (0, 3),
    'medium': (3, 6),
    'hard': (6, 9)
}

# sposób, w jaki wybieramy łamigłówki w każdym worku z danym poziomie trudności
# bottom - wybieramy pierwsze N z danego poziomu trudności (czyli najłatwiejsze z danego poziomu trudności)
# top - wybieramy ostatnie N z danego poziomu trudności (czyli najtrudniejsze z danego poziomu trudności)
# uniform - wybieramy N instancji z danego poziomu trudności równomiernie (dostaniemy trochę łatwiejszych i trochę trudniejszych)
selection_type = {
    'easy': 'top',
    'medium': 'top',
    'hard': 'top'
}

In [ ]:
raw_df.sort_values(['difficulty', 'clues'], ascending=[True, False])

Dzielenie na zbiory wg poziomu trudności (pokaże nam się ile instancji wpadło do jakiego poziomu trudności):

In [ ]:
intances_by_difficulty = {
    difficulty_name: raw_df.loc[(raw_df['difficulty'] >= left) & (raw_df['difficulty'] < right)].sort_values(['difficulty', 'clues'], ascending=[True, False])
    for (difficulty_name, (left, right)) in difficulty_ranges.items()
}
ninstances_by_difficulty = {diff_level: rows.shape[0] for diff_level, rows in intances_by_difficulty.items()}
print(ninstances_by_difficulty)

Teraz wybieramy tylko tyle ile nas interesuje z wybraną metodą (bottom/top/uniform) i dodatkowo zamieniamy stringi z instancjami na obiekty 2D.

In [ ]:
limited_instances = {}
for difficulty_name, instances in intances_by_difficulty.items():
    sampling_method = selection_type[difficulty_name]
    if sampling_method == 'bottom':
        limited_instances[difficulty_name] = intances_by_difficulty[difficulty_name].head(limit_per_difficulty_level)
    elif sampling_method == 'top':
        limited_instances[difficulty_name] = intances_by_difficulty[difficulty_name].tail(limit_per_difficulty_level)
    elif sampling_method == 'uniform':
        indices = [i * ninstances_by_difficulty[difficulty_name] // limit_per_difficulty_level for i in range(limit_per_difficulty_level)]
        limited_instances[difficulty_name] = intances_by_difficulty[difficulty_name].iloc(indices)
        
    # zamiana stringów na tablicę 2D
    to_2d_array = lambda puzzle: [list(map(int, puzzle[i:i+9])) for i in range(0, len(puzzle), 9)]
    limited_instances[difficulty_name].loc[:, 'puzzle'] = limited_instances[difficulty_name].loc[:, 'puzzle'].apply(to_2d_array)
    limited_instances[difficulty_name].loc[:, 'solution'] = limited_instances[difficulty_name].loc[:, 'solution'].apply(to_2d_array)

Inspekcja wybranych instancji:

In [ ]:
pprint({diff_level: rows.shape[0] for diff_level, rows in limited_instances.items()})

for difficulty_name, instances in limited_instances.items():
    print()
    print(difficulty_name)
    print(instances[['puzzle', 'difficulty', 'clues']])

## Stworzenie pliku json z instancjami

Tworzymy potężnego dicta z pandasowych dataframe'ów i zapisujemy go do pliku.

In [ ]:
limited_instances = {
    difficulty_name: instances.to_dict('records') for difficulty_name, instances in limited_instances.items()
}

with open('instances.json', 'w') as file:
    json.dump(limited_instances, file)